In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from spn.algorithms.Inference import log_likelihood
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.Base import Context
from spn.structure.leaves.parametric.Parametric import Categorical, Gaussian, Bernoulli
import numpy as np

newnames = ['telephoning',
 'medicine',
 'money',
 'traveling',
 'outside',
 'grocery',
 'cooking',
 'laundry',
 'light',
 'heavy',
 'toilet',
 'bathing',
 'dressing',
 'inside',
 'bed',
 'eating']

frame = pd.read_csv("nltcs/2to16disabilityNLTCS.txt", sep='\t').drop(['PERCENT'], axis=1)
frame = frame.rename({old: new  for old, new in zip(frame.columns, newnames)}, axis=1)
print ("%d rows" % len(frame))

D = frame.to_numpy()
expanded = []
for row in D:
    expanded.append([list(row[0:-1])] * row[-1])
D = np.concatenate(expanded, axis=0)
frame = pd.DataFrame(D, columns = frame.columns[:-1])
print ("%d rows" % len(frame))
frame.head()

3152 rows
21574 rows


,telephoning,medicine,money,traveling,outside,grocery,cooking,laundry,light,heavy,toilet,bathing,dressing,inside,bed,eating
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
import numpy as np

D = frame.to_numpy()
features = frame.columns.tolist()
categorical = [True]*len(features)
ptypes = [Bernoulli] * len(features)

train, test = train_test_split(D, random_state = 0)
print ("%d in train, %d in test" % (len(train), len(test)))
print (len(newnames))

16180 in train, 5394 in test
16


In [4]:
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.Base import Context

net = learn_parametric(train, 
                       ds_context = Context(parametric_types=ptypes).add_domains(train), 
                       rows = "gmm", ohe = True,
                       min_instances_slice = len(train) / 100)

In [6]:
import csi2
from spn.structure.Base import get_nodes_by_type, Product
names = features
def format_condition(condition):
    try:
        a, sign, b = condition.split(" ")
    except ValueError as e:
        raise ValueError("Some nodes don't have conditions. reduce min_impurity_decrease")
    
    if ((sign, b) == ('==', '1')) or ((sign, b) == ('!=', '0')):
        return ("%s" % (a))
    else:
        return ("~%s" % (a))

print ('#product nodes = ', len(get_nodes_by_type(net, (Product))))
net.n_instances = len(train)
csi2.annotate_spn(net, names, categorical, 
             min_impurity_decrease = 0.1, max_depth = 2)

rules = csi2.context_specific_independences(net, instance_threshold = 0)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

rules = csi2.context_specific_independences(net, instance_threshold = len(train) / 20,
                                           precision_threshold = 0.7, recall_threshold = 0.7)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

#product nodes =  74
74, 9.84, 3.32
19, 6.32, 4.05


In [9]:
from pprint import pprint 
for csi in csis:
    pprint(csi)

('{(~cooking) & (~dressing)} => {(cooking), (light), (dressing), (eating), '
 '(telephoning), (medicine), (money), '
 '(traveling,outside,grocery,laundry,heavy,toilet,bathing,inside,bed)} | 0.96, '
 '1.00, 10586 | 2 8')
('{[(cooking) | ((~cooking) & (dressing))] & [(bathing) & (grocery)]} => '
 '{(grocery), (heavy), (bathing), '
 '(telephoning,medicine,money,outside,toilet,dressing,inside,bed,eating), '
 '(traveling), (cooking,laundry,light)} | 0.99, 0.92, 3826 | 5 6')
('{[(~cooking) & (~dressing)] & [(~inside) & (~laundry)]} => {(laundry), '
 '(toilet), (inside), (bed), (traveling,grocery), (outside), (heavy), '
 '(bathing)} | 0.94, 1.00, 6907 | 4 8')
('{[(~cooking) & (~dressing)] & [(inside) | ((~inside) & (laundry))]} => '
 '{(traveling,outside,grocery,laundry,inside), (heavy), (toilet), (bathing), '
 '(bed)} | 0.96, 0.89, 3679 | 5 5')
('{[(cooking) | ((~cooking) & (dressing))] & [((bathing) & (~grocery)) | '
 '(~bathing)] & [~bathing]} => {(bathing), (eating), '
 '(telephoning,medi

In [44]:
from spn.algorithms.Inference import log_likelihood
import numpy as np

print ("test log-likelihood = %.2f" % np.mean(l-og_likelihood(net, test)))

test log-likelihood = -6.30
